<a href="https://colab.research.google.com/github/Rakesh-kumar-s/Rakesh-kumar-s/blob/main/Optimizing_Revenue_Leakage_and_Profitability_in_the_Hospitality_Sector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas sqlalchemy ipython-sql plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sqlalchemy import create_engine
%load_ext sql

In [3]:
# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')
%sql sqlite:///:memory:

In [4]:
# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:')
%sql sqlite:///:memory:

In [5]:
# Replace with your credentials
DB_HOST = "your-db-host"
DB_NAME = "your-db-name"
DB_USER = "your-username"
DB_PASS = "your-password"

# Create connection
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")
%sql postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/engine/base.py", line 145, in __init__
    self._dbapi_connection = engine.raw_connection()
                             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/engine/base.py", line 3297, in raw_connection
    return self.pool.connect()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/pool/base.py", line 449, in connect
    return _ConnectionFairy._checkout(self)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/pool/base.py", line 1264, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/pool/base.py", line 713, in checkout
    rec = pool._do_get()
          ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/pool/impl.py", lin

In [7]:
# Sample reservations data
reservations_data = {
    "reservation_id": [1, 2, 3, 4, 5],
    "guest_id": [101, 102, 103, 104, 105],
    "room_id": [201, 202, 203, 204, 205],
    "check_in_date": ["2023-10-01", "2023-10-02", "2023-10-03", "2023-10-04", "2023-10-05"],
    "check_out_date": ["2023-10-03", "2023-10-05", "2023-10-06", "2023-10-07", "2023-10-08"],
    "room_rate": [150, 200, 180, 220, 190],
    "booking_channel": ["Direct", "OTA", "Direct", "OTA", "Travel Agent"],
    "payment_status": ["completed", "completed", "cancelled", "completed", "pending"]
}

# Sample services data (ancillary revenue)
services_data = {
    "service_id": [1, 2, 3, 4, 5],
    "reservation_id": [1, 2, 3, 4, 5],
    "service_type": ["Spa", "Restaurant", "Bar", "Spa", "Tour"],
    "amount": [50, 30, 20, 60, 40]
}

# Convert to DataFrames and load into SQL
df_reservations = pd.DataFrame(reservations_data)
df_services = pd.DataFrame(services_data)

# Replace with your credentials
DB_HOST = "your-db-host" # <-- Replace with your database host
DB_NAME = "your-db-name" # <-- Replace with your database name
DB_USER = "your-username" # <-- Replace with your database username
DB_PASS = "your-password" # <-- Replace with your database password

# Create connection
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")
%sql postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}

df_reservations.to_sql("reservations", engine, index=False, if_exists="replace")
df_services.to_sql("services", engine, index=False, if_exists="replace")

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/engine/base.py", line 145, in __init__
    self._dbapi_connection = engine.raw_connection()
                             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/engine/base.py", line 3297, in raw_connection
    return self.pool.connect()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/pool/base.py", line 449, in connect
    return _ConnectionFairy._checkout(self)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/pool/base.py", line 1264, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/pool/base.py", line 713, in checkout
    rec = pool._do_get()
          ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sqlalchemy/pool/impl.py", lin

OperationalError: (psycopg2.OperationalError) could not translate host name "your-db-host" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
%%sql
SELECT
    booking_channel,
    COUNT(*) AS total_bookings,
    SUM(room_rate) AS potential_revenue,
    SUM(CASE WHEN payment_status = 'completed' THEN room_rate ELSE 0 END) AS realized_revenue,
    SUM(CASE WHEN payment_status != 'completed' THEN room_rate ELSE 0 END) AS leaked_revenue,
    ROUND((SUM(CASE WHEN payment_status != 'completed' THEN room_rate ELSE 0 END) * 100.0 / SUM(room_rate), 2) AS leakage_percentage
FROM reservations
GROUP BY booking_channel
ORDER BY leakage_percentage DESC;

In [ ]:
leakage_df = %sql SELECT booking_channel, leakage_percentage FROM (
    SELECT
        booking_channel,
        ROUND((SUM(CASE WHEN payment_status != 'completed' THEN room_rate ELSE 0 END) * 100.0 / SUM(room_rate)), 2) AS leakage_percentage
    FROM reservations
    GROUP BY booking_channel
)
leakage_df = leakage_df.DataFrame()

import plotly.express as px
fig = px.bar(
    leakage_df,
    x="booking_channel",
    y="leakage_percentage",
    title="Revenue Leakage by Booking Channel (%)",
    color="booking_channel"
)
fig.show()

In [ ]:
%%sql
SELECT
    STRFTIME('%w', check_in_date) AS day_of_week,
    AVG(room_rate) AS avg_rate,
    COUNT(*) AS bookings,
    SUM(room_rate) / COUNT(*) AS revenue_per_booking
FROM reservations
WHERE payment_status = 'completed'
GROUP BY day_of_week
ORDER BY revenue_per_booking DESC;

In [ ]:
%%sql
WITH daily_stats AS (
    SELECT
        check_in_date,
        COUNT(*) AS bookings,
        AVG(room_rate) AS avg_rate
    FROM reservations
    GROUP BY check_in_date
)
SELECT
    CASE
        WHEN bookings >= 3 THEN 'High Occupancy'
        WHEN bookings >= 1 THEN 'Medium Occupancy'
        ELSE 'Low Occupancy'
    END AS occupancy_level,
    AVG(avg_rate) AS avg_rate_achieved,
    COUNT(*) AS days_count
FROM daily_stats
GROUP BY occupancy_level
ORDER BY avg_rate_achieved DESC;

In [ ]:
%%sql
WITH daily_stats AS (
    SELECT
        check_in_date,
        COUNT(*) AS bookings,
        AVG(room_rate) AS avg_rate
    FROM reservations
    GROUP BY check_in_date
)
SELECT
    CASE
        WHEN bookings >= 3 THEN 'High Occupancy'
        WHEN bookings >= 1 THEN 'Medium Occupancy'
        ELSE 'Low Occupancy'
    END AS occupancy_level,
    AVG(avg_rate) AS avg_rate_achieved,
    COUNT(*) AS days_count
FROM daily_stats
GROUP BY occupancy_level
ORDER BY avg_rate_achieved DESC;

In [ ]:
%%sql
SELECT
    service_type,
    SUM(amount) AS total_revenue,
    COUNT(*) AS transactions,
    SUM(amount) / COUNT(*) AS avg_spend
FROM services
GROUP BY service_type
ORDER BY total_revenue DESC;

In [ ]:
%%sql
SELECT
    r.guest_id,
    SUM(r.room_rate) AS room_revenue,
    SUM(s.amount) AS ancillary_revenue,
    CASE
        WHEN SUM(s.amount) > 50 THEN 'High Spender'
        WHEN SUM(s.amount) > 20 THEN 'Medium Spender'
        ELSE 'Low Spender'
    END AS guest_segment
FROM reservations r
LEFT JOIN services s ON r.reservation_id = s.reservation_id
GROUP BY r.guest_id
ORDER BY ancillary_revenue DESC;

In [ ]:
ancillary_revenue_df = %sql SELECT * FROM (
    SELECT service_type, SUM(amount) AS revenue FROM services GROUP BY service_type
)
ancillary_revenue_df.DataFrame().to_csv("ancillary_revenue.csv", index=False)